VALIDATION WITHOUT FEATURE DROP

In [52]:
import pandas as pd
from matplotlib import pyplot
import numpy as np
import seaborn as sns
import math
import datetime
import pickle
import joblib

model_rfr_ftr = joblib.load('savemodel_rfr_noftr.sav')

model_dtr_ftr = joblib.load('savemodel_dtr_noftr.sav')


dfmain = pd.read_csv("AIS_weather_H_ok2_copy.csv",parse_dates=["Time"])
dfmain = dfmain[dfmain['LAT'] > 55.04 ]

dfpre = pd.read_csv("AIS_weather_h_rename_copy.csv",parse_dates=["Time"])
dfpre = dfpre[dfpre['LAT'] > 55.04 ]

In [53]:
dfmain = dfmain.drop(['Wind Speed','Current Speed'],axis=1)

In [54]:
dfpre = dfpre.drop(dfpre.columns[[0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,26]],axis=1)
dfpre = dfpre.rename({'Eastward wind': 'eastwind', 'Northward wind': 'northwind',
                           'Eastward current': 'eastcurrent', 'Northward current': 'northcurrent',
                           'SOG':'sog','COG':'cog','Heading':'heading'},axis=1) 
dfpre.head(n=5)

,eastwind,northwind,eastcurrent,northcurrent,sog,cog
0,2.267688,6.741482,0.113690,0.044994,0.0,358.9
1,1.782681,6.410275,0.113690,0.044994,0.0,358.9
2,1.154659,6.385135,0.083984,0.054000,0.0,358.9
3,0.348801,6.094659,0.083984,0.054000,0.0,358.9
4,-0.593128,6.153186,0.083984,0.054000,0.0,358.9


In [55]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=50)
imputer.fit(dfpre)

KNNImputer(n_neighbors=50)

In [56]:
x_pre = imputer.transform(dfpre)

In [57]:
dfpre = pd.DataFrame(x_pre, columns=dfpre.columns, index=dfpre.index)

In [58]:
dfpre["Current Speed"] = np.sqrt(dfpre["eastcurrent"]**2 + dfpre["northcurrent"]**2)
dfpre["Wind Speed"] = np.sqrt(dfpre["eastwind"]**2 + dfpre["northwind"]**2)

In [59]:
dfpre.describe()

,eastwind,northwind,eastcurrent,northcurrent,sog,cog,Current Speed,Wind Speed
count,7453.000000,7453.000000,7453.000000,7453.000000,7453.000000,7453.000000,7453.000000,7453.000000
mean,1.976344,0.379926,0.019228,0.004214,8.870751,242.699284,0.096788,6.513813
std,5.296850,4.429899,0.089903,0.074041,8.619512,108.555287,0.067697,3.048934
min,-15.216397,-12.824582,-0.491010,-0.404690,-0.009130,-0.134241,0.000735,0.193221
25%,-1.756650,-2.534061,-0.021486,-0.033055,0.000000,124.116508,0.047447,4.149883
50%,2.294543,0.385690,0.025485,0.003718,6.953578,283.216008,0.079148,6.285080
75%,5.838650,3.339456,0.062644,0.039626,18.014436,346.200000,0.128444,8.525129
max,16.338177,14.519458,0.522374,0.364966,21.070514,359.728115,0.525174,16.760001


In [60]:
dfpre["True North Wind Direction"] =  180 +  np.rad2deg(np.arctan((dfpre["eastwind"])/(dfpre["northwind"])))

# dfpre["True North Wind Direction"] = np.where(dfpre["True North Wind Direction"] > 0, 180 + dfpre["True North Wind Direction"] , 180 - dfpre["True North Wind Direction"])

# if dfpre["True North Wind Direction"]<0:
#     dfpre["True North Wind Direction"] = 180 - dfpre["True North Wind Direction"]
# elif dfpre["True North Wind Direction"]>0:
#     dfpre["True North Wind Direction"] = 180 - dfpre["True North Wind Direction"]

In [61]:
dfpre["True North Current Direction"] = np.rad2deg(np.arctan(dfpre["eastcurrent"]/dfpre["northcurrent"]))

In [62]:
dfpre = dfpre.drop(['eastwind','northwind','eastcurrent','northcurrent','sog','cog'],axis=1)

In [63]:
df = pd.concat([dfmain,dfpre],axis=1)
df.head()

,Unnamed: 0,Time,LON,LAT,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,...,Heading,True Wind Direction,True Current Direction,True Swell Direction,True Wind Wave Direction,True Wave Direction,Current Speed,Wind Speed,True North Wind Direction,True North Current Direction
0,0,2021-01-01 00:00:00,14.689663,55.098452,277.311035,1.259323,1.893614,4.818881,3.818459,4.154329,...,174.0,155.408180,105.591753,22.854050,22.749664,22.610062,0.122270,7.112664,198.591820,68.408246
1,1,2021-01-01 01:00:00,14.689659,55.098451,277.328369,1.259985,1.752303,4.755800,3.470216,4.130580,...,174.0,158.458891,105.591753,26.122772,19.152008,22.209274,0.122270,6.653538,195.541109,68.408246
2,2,2021-01-01 02:00:00,14.689648,55.098468,277.390625,1.260244,1.619775,4.645083,3.259682,4.083755,...,174.0,163.749663,116.740448,28.498383,12.424912,20.857971,0.099846,6.488697,190.250337,57.259552
3,3,2021-01-01 03:00:00,14.689646,55.098471,277.411377,1.260148,1.518736,4.588384,3.177906,4.010502,...,174.0,170.724504,116.740448,29.956985,6.725922,18.732071,0.099846,6.104632,183.275496,57.259552
4,4,2021-01-01 04:00:00,14.689644,55.098462,277.430664,1.260634,1.455034,4.528841,3.281654,3.913362,...,174.0,179.505940,116.740448,34.486298,3.761627,15.731430,0.099846,6.181707,174.494060,57.259552


In [64]:
# SOG threshold according to Abebe
df = df[df['SOG'] > 5 ]

In [65]:
# Exclude data for the month of June to check the model's forecasting performance 

dfdate6 = df[df['Time'].dt.strftime('%Y-%m') == '2021-06']
df = df[df['Time'].dt.strftime('%Y-%m') != '2021-06']

# Extract information of a journey from a single day

koege_dep = df[df['Time'].dt.strftime('%Y-%m-%d %H:%M:%S') == '2021-05-31 23:00:00']
koege_arr = dfdate6[dfdate6['Time'].dt.strftime('%Y-%m-%d') == '2021-06-01']

# Drop the departure time for next trip

koege_arr2 = koege_arr.drop(koege_arr.index[14])

# To gain information for a single trip from Koege to Ronne

ronne_arr = koege_arr.drop(koege_arr.index[4:15])

dfday6 = pd.concat([koege_dep,koege_arr2])
dfone6 = pd.concat([koege_dep,ronne_arr])

# # For Qgis Single Journey
# dfday6.to_csv("AIS_01_06_21_koegeronne.csv")

Special Section

In [66]:
# # Check model performance for one trip
# # PLEASE COMMMENT WHEN DONE !!!

# dfdate6 = dfone6

Measure performance in one month

In [67]:
dfdate6 = dfdate6.drop(['Unnamed: 0','Time','LON','LAT'],axis=1)
dfdate6.head(n=5)

,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,Sea surface temperature,Combined wind waves and swell height,Swell height,Wind wave height,...,Heading,True Wind Direction,True Current Direction,True Swell Direction,True Wind Wave Direction,True Wave Direction,Current Speed,Wind Speed,True North Wind Direction,True North Current Direction
119,275.823975,1.283481,3.358080,6.738089,4.925452,5.291555,278.661621,1.759765,0.743164,1.583740,...,100.782313,100.989288,164.871808,15.554049,66.843493,57.268527,0.144914,10.248966,201.771601,85.654121
120,275.924011,1.283425,3.578120,7.185143,5.342444,5.697814,278.936035,1.950242,0.841925,1.757568,...,96.934816,108.737155,153.650411,11.071672,60.455881,51.313932,0.246220,11.364678,205.671971,70.585227
121,276.285034,1.279012,4.184914,7.260825,5.460486,6.126474,279.209961,2.189788,0.933936,1.910706,...,97.525758,114.910729,153.031589,16.281358,57.086511,46.894106,0.284315,12.666946,212.436488,70.557348
122,277.189484,1.269904,4.908680,6.808745,6.225745,6.284671,279.216553,2.565230,0.723303,2.452637,...,106.941521,123.568984,153.643227,69.175152,55.507286,56.527939,0.300428,13.386999,230.510505,80.584748
126,277.408752,1.263999,4.621826,6.813305,5.970984,6.291378,279.319824,2.414834,1.022894,2.130981,...,227.680837,26.476633,41.766758,158.632374,159.050718,167.192888,0.262897,12.053293,254.157471,89.447596


In [68]:
# dfdate6= dfdate6.rename({'Max wave height': 'waveheight', 'Draught': 'draught',
#                            'SOG': 'sog', 'Wind Speed': 'windspeed', 
#                            'True Wind Direction': 'truewinddir','Temperature above oceans' : 'oceantemperature',
#                            'COG': 'cog', 'Current Speed' : 'curspeed','True Wave Direction' : 'truewavedir',
#                             'Swell period': 'swellperiod','Wind wave period': 'windwaveperiod','Sea surface temperature': 'surftemp',
#                             'Combined wind waves and swell height': 'windwaveswellheight','Swell height': 'swellheight','Wind wave height': 'windwaveheight',
#                             'Heading': 'heading','True Current Direction': 'truecurrentdir','True Swell Direction': 'trueswelldir',
#                             'True Wind Wave Direction': 'truewindwavedir','Wave period': 'waveperiod',
#                             'True North Wind Direction' : 'truenorthwinddir' , 'True North Current Direction' : 'truenorthcurrentdir'
#                            }, axis=1) 

In [69]:
# Rearrange column to match model 
# Also dropping the features as described in the main script

# dfdate6 = dfdate6[['oceantemperature','waveperiod','surftemp','windwaveswellheight','draught','sog','cog','heading','windspeed','curspeed','truewinddir','truecurrentdir','truewavedir', 'truenorthwinddir' , 'truenorthcurrentdir']]


# dfdate6 = dfdate6[['oceantemperature','waveheight','swellperiod','windwaveperiod','waveperiod','surftemp','windwaveswellheight','swellheight','windwaveheight','draught','sog','cog','heading','windspeed','curspeed','truewinddir','truecurrentdir','trueswelldir','truewindwavedir','truewavedir', 'truenorthwinddir' , 'truenorthcurrentdir']]

In [70]:
# stats2 = dfdate6['curspeed'].value_counts()

In [71]:
import numpy as np
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=50)


In [72]:
imputer.fit(dfdate6)

KNNImputer(n_neighbors=50)

In [73]:
xdate = imputer.transform(dfdate6)

In [74]:
dfdate6tr = pd.DataFrame(xdate, columns=dfdate6.columns, index=dfdate6.index)

In [75]:
x_date = dfdate6tr.drop(['SOG'],axis=1)
y_date = dfdate6tr.SOG

In [76]:
def predict_y(x_test,model_type):
    y_predicted = model_type.predict(x_test)
    return y_predicted

def display_scores(x_test,y_test,model_type,y_predicted):
    from sklearn.metrics import explained_variance_score,mean_absolute_error,mean_squared_error,median_absolute_error
    print("R^2 score (Indicate model fit. Best Score = 1):", model_type.score(x_test,y_test))
    print("Explained Variance EV (Indicate amount of variance in model. Best Score = 1):", explained_variance_score(y_test,y_predicted))
    print("Mean Absolute Error MAE (Indicate how much error a model makes in its prediction. Best Score = 0):", mean_absolute_error(y_test,y_predicted))
    print("Root Mean Square Error RMSE (Same as MAE, more sensitive to outlier. Best Score = 0):", mean_squared_error(y_test,y_predicted))
    print("Median Absolute Error MAD (Check robustness against outlier. Best Score = 1):", median_absolute_error(y_test,y_predicted))

y_pred_rfr_ftr = predict_y(x_date,model_rfr_ftr)
display_scores(x_date,y_date,model_rfr_ftr,y_pred_rfr_ftr)

R^2 score (Indicate model fit. Best Score = 1): 0.8621336226422058
Explained Variance EV (Indicate amount of variance in model. Best Score = 1): 0.8815890891126791
Mean Absolute Error MAE (Indicate how much error a model makes in its prediction. Best Score = 0): 0.8285334679048141
Root Mean Square Error RMSE (Same as MAE, more sensitive to outlier. Best Score = 0): 1.2952942181859202
Median Absolute Error MAD (Check robustness against outlier. Best Score = 1): 0.634867336150001


In [77]:
# # Prediction using FTR + HPO

# y_pred_rfr_hpo = predict_y(x_date,model_rfr_hpo)
# display_scores(x_date,y_date,model_rfr_hpo,y_pred_rfr_hpo)


In [78]:
# Comparing performance with DTR model (not optimzed)

y_pred_dtr_ftr = predict_y(x_date,model_dtr_ftr)
display_scores(x_date,y_date,model_dtr_ftr,y_pred_dtr_ftr)

R^2 score (Indicate model fit. Best Score = 1): 0.7721707856597468
Explained Variance EV (Indicate amount of variance in model. Best Score = 1): 0.7959853961221279
Mean Absolute Error MAE (Indicate how much error a model makes in its prediction. Best Score = 0): 1.0641863158602487
Root Mean Square Error RMSE (Same as MAE, more sensitive to outlier. Best Score = 0): 2.1405209139782118
Median Absolute Error MAD (Check robustness against outlier. Best Score = 1): 0.8372708949999996


In [79]:
# # Comparing performacw with MLR as baseline (not optimzed)

# y_pred_mlr_ftr = predict_y(x_date,model_mlr_ftr)
# display_scores(x_date,y_date,model_mlr_ftr,y_pred_mlr_ftr)

In [81]:
ydate_pred = y_pred_rfr_ftr
dfprog_sog = pd.Series(y_date,name="sog_act").to_frame()
dfprog_sog["sog_pred"] = ydate_pred
dfprog_sog["gamma"] = dfdate6tr["True North Current Direction"]
dfprog_sog["nwinddir"] = dfdate6tr["True North Wind Direction"]

print(dfprog_sog)

        sog_act   sog_pred      gamma    nwinddir
119   18.998906  18.474639  85.654121  201.771601
120   19.334278  18.519364  70.585227  205.671971
121   18.911047  18.514665  70.557348  212.436488
122   13.854933  13.888061  80.584748  230.510505
126    9.305198  10.931940  89.447596  254.157471
...         ...        ...        ...         ...
7413  19.072622  18.334391 -75.033834  110.959927
7414  17.696584  17.821903  86.330174  126.837339
7415  15.963630  16.476884  83.486098  119.018621
7418   8.988086   8.734655  89.745593  140.723377
7419  17.672769  17.353285  15.792214  143.902554

[322 rows x 4 columns]


In [82]:
dfprog = pd.concat([x_date,dfprog_sog],axis=1)
dfprog.head(n=10)

,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,Sea surface temperature,Combined wind waves and swell height,Swell height,Wind wave height,...,True Wind Wave Direction,True Wave Direction,Current Speed,Wind Speed,True North Wind Direction,True North Current Direction,sog_act,sog_pred,gamma,nwinddir
119,275.823975,1.283481,3.358080,6.738089,4.925452,5.291555,278.661621,1.759765,0.743164,1.583740,...,66.843493,57.268527,0.144914,10.248966,201.771601,85.654121,18.998906,18.474639,85.654121,201.771601
120,275.924011,1.283425,3.578120,7.185143,5.342444,5.697814,278.936035,1.950242,0.841925,1.757568,...,60.455881,51.313932,0.246220,11.364678,205.671971,70.585227,19.334278,18.519364,70.585227,205.671971
121,276.285034,1.279012,4.184914,7.260825,5.460486,6.126474,279.209961,2.189788,0.933936,1.910706,...,57.086511,46.894106,0.284315,12.666946,212.436488,70.557348,18.911047,18.514665,70.557348,212.436488
122,277.189484,1.269904,4.908680,6.808745,6.225745,6.284671,279.216553,2.565230,0.723303,2.452637,...,55.507286,56.527939,0.300428,13.386999,230.510505,80.584748,13.854933,13.888061,80.584748,230.510505
126,277.408752,1.263999,4.621826,6.813305,5.970984,6.291378,279.319824,2.414834,1.022894,2.130981,...,159.050718,167.192888,0.262897,12.053293,254.157471,89.447596,9.305198,10.931940,89.447596,254.157471
127,277.399658,1.265251,3.852527,6.907508,5.299354,6.061778,279.230591,2.021695,1.143225,1.607422,...,126.285782,119.939682,0.181460,11.667020,252.289134,85.735437,20.935741,17.309680,85.735437,252.289134
128,275.499359,1.266485,3.304548,6.816074,4.561194,5.927387,289.276207,1.754722,1.136414,1.240662,...,173.337517,176.253205,0.281402,9.043038,240.000273,87.870444,12.419853,12.926472,87.870444,240.000273
130,276.370789,1.267139,2.999140,6.680770,3.821632,5.959699,289.294116,1.585114,1.306714,0.837463,...,59.314576,54.778275,0.185268,8.393962,248.405148,-84.704755,20.447721,16.718160,-84.704755,248.405148
131,277.268555,1.267037,3.283719,6.891045,4.686806,5.876345,279.216553,1.731217,1.167188,1.231018,...,24.484072,41.385363,0.133047,9.328641,264.710374,82.800757,18.702889,16.885598,82.800757,264.710374
134,277.220703,1.265203,2.827531,6.624643,4.262856,5.585643,279.216553,1.490863,1.038464,1.032349,...,178.761731,159.775304,0.139192,9.022607,269.075598,-78.378834,16.155258,15.534029,-78.378834,269.075598


In [83]:
# Conversion of predicted SOG to m/s

dfprog["vgms"] = dfprog["sog_pred"]/1.9438

# Conversion of actual SOG to m/s

dfprog["vgms_act"] = dfprog["sog_act"]/1.9438

In [85]:
# Conversion of the angles to radian

rad_gamma = np.deg2rad(dfprog["gamma"])
rad_cog = np.deg2rad(dfprog["COG"])

In [86]:
# Calculation of the predicted x-component of SOG

dfprog["vgx"] = dfprog["vgms"] * np.sin(rad_cog)
dfprog["vcx"] = dfprog["Current Speed"] * np.sin(rad_gamma)
dfprog["stw_x"] = (dfprog["vgx"] - dfprog["vcx"])

# Calculation of the actual x-component of SOG

dfprog["vgx_act"] = dfprog["vgms_act"] * np.sin(rad_cog)
dfprog["stw_x_act"] = (dfprog["vgx_act"] - dfprog["vcx"])


In [87]:
# Calculation of the predicted y-component of SOG 

dfprog["vgy"] = dfprog["vgms"] * np.cos(rad_cog)
dfprog["vcy"] = dfprog["Current Speed"] * np.cos(rad_gamma)
dfprog["stw_y"] = (dfprog["vgy"] - dfprog["vcy"])

# Calculation of the actual y-component of SOG

dfprog["vgy_act"] = dfprog["vgms_act"] * np.cos(rad_cog)
dfprog["stw_y_act"] = (dfprog["vgy_act"] - dfprog["vcy"])


In [88]:
# For the actual data
dfprog["vwms_a"] = np.sqrt(dfprog["stw_x_act"]**2 + dfprog["stw_y_act"]**2)
dfprog["stw_act"] = dfprog["vwms_a"]*1.9438

# For the prediction

dfprog["vwms_p"] = np.sqrt(dfprog["stw_x"]**2 + dfprog["stw_y"]**2)
dfprog["stw_pred"] = dfprog["vwms_p"]*1.9438



In [89]:
dfprog = dfprog.drop(['vgms','vgx','vcx','stw_x',
                      'vgy','vcy','stw_y',
                      'vgms_act','vgx_act','stw_x_act',
                      'vgy_act','stw_y_act',
                      'vwms_p','vwms_a'],axis=1)
#df_ship.head(n=5)
dfprog.head(n=10)

,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,Sea surface temperature,Combined wind waves and swell height,Swell height,Wind wave height,...,Current Speed,Wind Speed,True North Wind Direction,True North Current Direction,sog_act,sog_pred,gamma,nwinddir,stw_act,stw_pred
119,275.823975,1.283481,3.358080,6.738089,4.925452,5.291555,278.661621,1.759765,0.743164,1.583740,...,0.144914,10.248966,201.771601,85.654121,18.998906,18.474639,85.654121,201.771601,18.729972,18.205711
120,275.924011,1.283425,3.578120,7.185143,5.342444,5.697814,278.936035,1.950242,0.841925,1.757568,...,0.246220,11.364678,205.671971,70.585227,19.334278,18.519364,70.585227,205.671971,18.913459,18.098604
121,276.285034,1.279012,4.184914,7.260825,5.460486,6.126474,279.209961,2.189788,0.933936,1.910706,...,0.284315,12.666946,212.436488,70.557348,18.911047,18.514665,70.557348,212.436488,18.429833,18.033494
122,277.189484,1.269904,4.908680,6.808745,6.225745,6.284671,279.216553,2.565230,0.723303,2.452637,...,0.300428,13.386999,230.510505,80.584748,13.854933,13.888061,80.584748,230.510505,13.276732,13.309860
126,277.408752,1.263999,4.621826,6.813305,5.970984,6.291378,279.319824,2.414834,1.022894,2.130981,...,0.262897,12.053293,254.157471,89.447596,9.305198,10.931940,89.447596,254.157471,9.009772,10.635086
127,277.399658,1.265251,3.852527,6.907508,5.299354,6.061778,279.230591,2.021695,1.143225,1.607422,...,0.181460,11.667020,252.289134,85.735437,20.935741,17.309680,85.735437,252.289134,21.221281,17.595431
128,275.499359,1.266485,3.304548,6.816074,4.561194,5.927387,289.276207,1.754722,1.136414,1.240662,...,0.281402,9.043038,240.000273,87.870444,12.419853,12.926472,87.870444,240.000273,12.753846,13.260176
130,276.370789,1.267139,2.999140,6.680770,3.821632,5.959699,289.294116,1.585114,1.306714,0.837463,...,0.185268,8.393962,248.405148,-84.704755,20.447721,16.718160,-84.704755,248.405148,20.747280,17.017934
131,277.268555,1.267037,3.283719,6.891045,4.686806,5.876345,279.216553,1.731217,1.167188,1.231018,...,0.133047,9.328641,264.710374,82.800757,18.702889,16.885598,82.800757,264.710374,18.470818,16.653565
134,277.220703,1.265203,2.827531,6.624643,4.262856,5.585643,279.216553,1.490863,1.038464,1.032349,...,0.139192,9.022607,269.075598,-78.378834,16.155258,15.534029,-78.378834,269.075598,15.886581,15.265353


In [90]:
dfprog.describe()

,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,Sea surface temperature,Combined wind waves and swell height,Swell height,Wind wave height,...,Current Speed,Wind Speed,True North Wind Direction,True North Current Direction,sog_act,sog_pred,gamma,nwinddir,stw_act,stw_pred
count,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,3.220000e+02,...,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000,322.000000
mean,287.133146,1.223464,1.246392,3.741467,3.217393,3.404828,287.291491,0.660399,0.330605,5.063153e-01,...,0.092478,5.832751,179.967365,-11.849461,17.449699,17.022160,-11.849461,179.967365,17.440998,17.013472
std,6.306975,0.030251,1.057415,1.141750,0.965232,0.964344,4.814228,0.542658,0.191253,5.569619e-01,...,0.068917,3.020207,63.036318,55.353826,3.069944,2.765370,55.353826,63.036318,3.059230,2.747118
min,269.606903,1.180081,0.123628,1.915932,1.570717,1.915993,275.485352,0.073242,0.073242,2.530000e-16,...,0.002006,0.441768,90.107133,-89.500098,5.224523,5.360733,-89.500098,90.107133,5.211211,5.395379
25%,285.292786,1.200682,0.488767,2.886734,2.440019,2.692099,286.832031,0.272528,0.211311,1.016693e-01,...,0.043318,3.442231,119.047681,-60.671287,16.568861,16.667360,-60.671287,119.047681,16.581974,16.604441
50%,289.505615,1.216316,0.851829,3.536526,3.121132,3.201987,289.295374,0.457853,0.286448,3.130442e-01,...,0.073482,5.528616,170.499216,-26.065061,18.413643,18.081722,-26.065061,170.499216,18.399973,17.988737
75%,291.391602,1.235258,1.500434,4.301386,3.864905,3.764453,290.459228,0.788779,0.391537,6.430664e-01,...,0.123647,7.569939,244.414903,35.032562,19.421633,18.580268,35.032562,244.414903,19.417551,18.578046
max,294.685303,1.323879,4.908680,7.260825,6.225745,6.291378,292.665649,2.565230,1.306714,2.452637e+00,...,0.417155,13.386999,269.737370,89.975398,21.031230,19.560689,89.975398,269.737370,21.221281,19.654261


In [91]:
# FOC in T/d
dfprog["foc_act_d"] = ((dfprog["stw_act"]/17.7)**3) * 21.14112
dfprog["foc_pred_d"] = ((dfprog["stw_pred"]/17.7)**3) * 21.14112

# FOC in T/h
dfprog["foc_act_h"] = ((dfprog["stw_act"]/17.7)**3) * 0.8808
dfprog["foc_pred_h"] = ((dfprog["stw_pred"]/17.7)**3) * 0.8808

In [92]:
dfprog.head(n=10)

,Temperature above oceans,Air density above oceans,Max wave height,Swell period,Wind wave period,Wave period,Sea surface temperature,Combined wind waves and swell height,Swell height,Wind wave height,...,sog_act,sog_pred,gamma,nwinddir,stw_act,stw_pred,foc_act_d,foc_pred_d,foc_act_h,foc_pred_h
119,275.823975,1.283481,3.358080,6.738089,4.925452,5.291555,278.661621,1.759765,0.743164,1.583740,...,18.998906,18.474639,85.654121,201.771601,18.729972,18.205711,25.050684,23.005471,1.043684,0.958474
120,275.924011,1.283425,3.578120,7.185143,5.342444,5.697814,278.936035,1.950242,0.841925,1.757568,...,19.334278,18.519364,70.585227,205.671971,18.913459,18.098604,25.794140,22.601822,1.074658,0.941657
121,276.285034,1.279012,4.184914,7.260825,5.460486,6.126474,279.209961,2.189788,0.933936,1.910706,...,18.911047,18.514665,70.557348,212.436488,18.429833,18.033494,23.865601,22.358767,0.994310,0.931531
122,277.189484,1.269904,4.908680,6.808745,6.225745,6.284671,279.216553,2.565230,0.723303,2.452637,...,13.854933,13.888061,80.584748,230.510505,13.276732,13.309860,8.922402,8.989358,0.371733,0.374523
126,277.408752,1.263999,4.621826,6.813305,5.970984,6.291378,279.319824,2.414834,1.022894,2.130981,...,9.305198,10.931940,89.447596,254.157471,9.009772,10.635086,2.788365,4.585969,0.116171,0.191065
127,277.399658,1.265251,3.852527,6.907508,5.299354,6.061778,279.230591,2.021695,1.143225,1.607422,...,20.935741,17.309680,85.735437,252.289134,21.221281,17.595431,36.435353,20.768635,1.518002,0.865281
128,275.499359,1.266485,3.304548,6.816074,4.561194,5.927387,289.276207,1.754722,1.136414,1.240662,...,12.419853,12.926472,87.870444,240.000273,12.753846,13.260176,7.909183,8.889064,0.329519,0.370344
130,276.370789,1.267139,2.999140,6.680770,3.821632,5.959699,289.294116,1.585114,1.306714,0.837463,...,20.447721,16.718160,-84.704755,248.405148,20.747280,17.017934,34.048008,18.790082,1.418538,0.782849
131,277.268555,1.267037,3.283719,6.891045,4.686806,5.876345,279.216553,1.731217,1.167188,1.231018,...,18.702889,16.885598,82.800757,264.710374,18.470818,16.653565,24.025177,17.608803,1.000958,0.733633
134,277.220703,1.265203,2.827531,6.624643,4.262856,5.585643,279.216553,1.490863,1.038464,1.032349,...,16.155258,15.534029,-78.378834,269.075598,15.886581,15.265353,15.286199,13.562156,0.636867,0.565038


In [94]:
# df_foc=dfprog.drop(['oceantemperature','waveheight','swellperiod','windwaveperiod','waveperiod',
#                     'surftemp','windwaveswellheight','swellheight','windwaveheight','swellheight',
#                     'windwaveheight','draught','cog','heading','windspeed','curspeed','truewinddir','truecurrentdir','trueswelldir','truewindwavedir',
#                     'truewavedir','gamma','nwinddir'],axis=1)

# df_foc=dfprog.drop(['oceantemperature','waveperiod',
#                     'surftemp','windwaveswellheight',
#                     'draught','cog','heading','windspeed','curspeed','truewinddir','truecurrentdir',
#                     'truewavedir','gamma','nwinddir'],axis=1)

In [95]:
# import matplotlib.pyplot as plt
# plt.rcParams.update(plt.rcParamsDefault)
# df_foc.hist(bins=50,figsize=(20,15))
# plt.show()

In [96]:
foc_month = dfprog['foc_pred_d'].mean()
foc_month_a = dfprog['foc_act_d'].mean()
print(f"Mean of predicted FOC in a month is {foc_month} T/d")
print(f"Mean of actual FOC in a month is {foc_month_a} T/d")

Mean of predicted FOC in a month is 20.053153403696147 T/d
Mean of actual FOC in a month is 21.87772146283159 T/d


In [97]:
foc_trip = dfprog['foc_pred_h'].mean() * 4
foc_trip_a = dfprog['foc_act_h'].mean() * 4
foc_trip_err = foc_trip_a - foc_trip
print(f"Mean of predicted FOC in a 4h trip is {foc_trip} T")
print(f"Mean of actual FOC in a trip in a 4h trip is {foc_trip_a} T")
print(f"Error is {foc_trip_err} T")

Mean of predicted FOC in a 4h trip is 3.341888701823852 T
Mean of actual FOC in a trip in a 4h trip is 3.6459557609931856 T
Error is 0.30406705916933374 T


In [98]:
haha

NameError: name 'haha' is not defined

Hyperparameter Optimization

In [ ]:
# from pprint import pprint
# pprint(model_rfr_ftr.get_params())

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}


In [ ]:
# from sklearn.model_selection import RandomizedSearchCV
# # Number of trees in random forest
# n_estimators = [int(x) for x in np.linspace(start = 10, stop = 300, num = 10)]
# # Number of features to consider at every split
# max_features = [int(x) for x in np.linspace(1, 12, num = 11)]
# # Maximum number of levels in tree
# max_depth = [int(x) for x in np.linspace(10, 200, num = 10)]
# max_depth.append(None)
# # Minimum number of samples required to split a node
# min_samples_split = [2, 5, 10]
# # Minimum number of samples required at each leaf node
# min_samples_leaf = [1, 2, 4]
# # Method of selecting samples for training each tree
# bootstrap = [True, False]# Create the random grid
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}
# pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200, None],
 'max_features': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10, 42, 74, 106, 138, 171, 203, 235, 267, 300]}


In [ ]:
# rf_random = RandomizedSearchCV(estimator = model_rfr_ftr, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
# rf_random.fit(x_date, y_date)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 31, 52, 73, 94, 115,
                                                      136, 157, 178, 200,
                                                      None],
                                        'max_features': [1, 2, 3, 4, 5, 6, 7, 8,
                                                         9, 10, 12],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 42, 74, 106, 138,
                                                         171, 203, 235, 267,
                                                         300]},
                   random_state=42, verbose=2)

In [ ]:
# rf_random.best_params_

{'n_estimators': 235,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 7,
 'max_depth': 52,
 'bootstrap': False}